# MSBX 5420 Assignment 2
## Task 1 - Warm Up with Spark

### Replicate the MapReduce Calculations in Assignment 1 with Spark RDD, DataFrame and SQL
As a warm-up, your first task will be replicating the same MapReduce jobs in assignment 1 with spark RDD and DataFrame/SQL.
First, let's load the NFL dataset into Spark. For convenience we will use spark session and dataframe.

In [ ]:
from pyspark.sql import SparkSession

#let's use all cores with 'local[*]' to somehow speed up
spark = SparkSession.builder.master('local[*]').appName('spark_nfl_data').getOrCreate()

If you look at the NFL dataset, those missing values we have tried to skip in assignment 1 are "NA". By default, spark will treat empty or null as missing values, so here we need to let spark treat "NA" as the missing values.

In [ ]:
df_nfl = spark.read.options(header=True, nullValue='NA', inferSchema=True).csv('./NFL_Play_by_Play_2009-2018.csv')
df_nfl.show(5)

In [ ]:
#let's check and clean the data with dataframe
print(df_nfl.count())
print(df_nfl.distinct().count())

In [ ]:
df_nfl = df_nfl.dropDuplicates()

In [ ]:
#let's look at the data
df_nfl.printSchema()

In [ ]:
#take a look at the dta via pandas
import pandas as pd
#disable the row/column limits to not truncate the displayed data
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_nfl.limit(10).toPandas().head(10)

In [ ]:
#let's also see the number of partitions
df_nfl.rdd.getNumPartitions()

In [ ]:
#convert the dataset to a RDD
rdd_nfl = df_nfl.rdd
rdd_nfl.take(5)

Now we have the RDD converted from Dataframe so we can do RDD operations with `rdd_nfl`. Now let's replicate the two calculations (1) number of plays in each game (2) average yarns gained in each game.

In [ ]:
#mapreduce with spark RDD for sum of plays
#[Your Code]
nfl_play_sum = rdd_nfl.map(lambda row: (row['game_id'], 1)).reduceByKey(lambda x, y: x+y)
nfl_play_sum.take(10)

In [ ]:
#mapreduce with spark RDD for average yards gained
#hint: there are "NA"s (None) in the column "yards_gained"; after into dataframe and transformed as RDD, "NA" is None now, and others are integers
#[Your Code]
nfl_yard_avg = rdd_nfl.filter(lambda row: row['yards_gained'] != None).map(lambda row: (row['game_id'], row['yards_gained'])).groupByKey().mapValues(lambda x: sum(x) / len(x))
nfl_yard_avg.take(10)

Next, let's do it with spark dataframe and SQL. The dataframe is `df_nfl`.

In [ ]:
#use dataframe operations/api
#[Your Code]
df_nfl.groupBy('game_id').count().show()
df_nfl.groupBy('game_id').avg('yards_gained').show()

In [ ]:
#use spark sql - don't forget to create temp view before querying the dataframe
#[Your Code]
df_nfl.createOrReplaceTempView('df_nfl')
spark.sql('select game_id, count(*) from df_nfl group by game_id').show()
spark.sql('select game_id, avg(yards_gained) from df_nfl group by game_id').show()

## Task 2 - Data Analytics with Spark DataFrame and SQL
### Answer four data analytics questions on NFL dataset to solve the problems
With the NFL Dataframe `df_nfl`, use either dataframe operations/API or spark SQL to answer the following questions.
First of all, let's build a data viewer to look at the data so we can understand the values better.

In [ ]:
#build a data viewer to check data for a game; the list basically contains all the columns to use in this task
game_info_all = ['play_id', 'game_id', 'home_team', 'away_team', 'game_date', 
                 'posteam', 'posteam_type', 'defteam',
                 'total_home_score', 'total_away_score',
                 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown']
#df_nfl.select(game_info).limit(200).toPandas().head(200)
df_nfl.select(game_info_all).where('game_id = 2018111110').toPandas().head(200)

Now you are going to answer the following questions using spark dataframe or spark SQL. You can choose either one to solve the problem and output results.
1. Which game(s) has the highest number of plays from 2009 to 2018? And which game has the highest final score difference?

In [ ]:
from pyspark.sql import functions as fn
from pyspark.sql import Window

#you need to show the game info with the highest plays, so let's obtain game level information
game_info = ['game_id', 'home_team', 'away_team', 'game_date', 'total_home_score', 'total_away_score']
#because we need the final scores for each game as game level info, we can do that by filtering the maxiumn play id to get game level info
window = Window.partitionBy('game_id')
nfl_game_info = df_nfl.withColumn("max_play_id", fn.max("play_id").over(window)).filter("max_play_id = play_id").drop("max_play_id").select(game_info)
nfl_game_info.show()

In [ ]:
#DataFrame API Solution
#get number of plays in each game
nfl_num_play = df_nfl.groupBy('game_id').agg(fn.count('play_id').alias('num_plays'))
nfl_num_play.show()

#join the two dataframes
nfl_game_info = nfl_game_info.join(nfl_num_play, 'game_id')

#[Your Code] to get the game with highest number of plays
win = Window.partitionBy()
nfl_game_info.withColumn("max_plays", fn.max("num_plays").over(win)).filter("max_plays = num_plays").drop("max_plays").show()

In [ ]:
#DataFrame API Solution
#now it is the score difference
nfl_game_info = nfl_game_info.withColumn('score_diff', fn.abs(nfl_game_info['total_home_score'] - nfl_game_info['total_away_score']))

#[Your Code] to get the game with highest score difference
win = Window.partitionBy()
nfl_game_info.withColumn("max_score_diff", fn.max("score_diff").over(win)).filter("max_score_diff = score_diff").drop("max_score_diff").show()

In [ ]:
#Spark SQL Solution
#max plays
nfl_num_play = spark.sql('select game_id, count(*) as num_plays from df_nfl group by game_id')

nfl_game_info.createOrReplaceTempView('nfl_game_info')
nfl_num_play.createOrReplaceTempView('nfl_num_play')

nfl_game_info = spark.sql('select a.*, b.num_plays from nfl_game_info a, nfl_num_play b where a.game_id = b.game_id')

#[Your Code] to get the game with highest number of plays
nfl_game_info.createOrReplaceTempView('nfl_game_info')
spark.sql('select * from nfl_game_info where num_plays = (select max(num_plays) from nfl_game_info)').show()

In [ ]:
#Spark SQL Solution
#max score diff
nfl_game_info = spark.sql('select *, abs(total_home_score-total_away_score) as score_diff from nfl_game_info')

#[Your Code] to get the game with highest number of plays
nfl_game_info.createOrReplaceTempView('nfl_game_info')
spark.sql('select * from nfl_game_info where score_diff = (select max(score_diff) from nfl_game_info)').show()

2. On average how many plays are needed for a successful touchdown, for home team and away team, respectively?

In [ ]:
#DataFrame API Solution
nfl_game_play = df_nfl.groupBy('game_id').agg(fn.count('play_id').alias('total_plays'), fn.sum('touchdown').alias('total_touchdowns'))
#[Your Code] to take average for total_plays/total_touchdowns
nfl_game_play.groupby().agg(fn.avg(fn.col('total_plays')/fn.col('total_touchdowns'))).show()

nfl_team_play = df_nfl.groupBy('game_id', 'posteam_type').agg(fn.count('play_id').alias('total_plays'), fn.sum('touchdown').alias('total_touchdowns'))
#[Your Code] to take average for total_plays/total_touchdowns by posteam_type
nfl_team_play.groupby('posteam_type').agg(fn.avg(fn.col('total_plays')/fn.col('total_touchdowns'))).show()

In [ ]:
#Spark SQL Solution
df_nfl.createOrReplaceTempView('df_nfl')

nfl_game_play = spark.sql('select game_id, count(*) as total_plays, \
                          sum(touchdown) as total_touchdowns from df_nfl group by game_id')

#[Your Code] to take average for total_plays/total_touchdowns
nfl_game_play.createOrReplaceTempView('nfl_game_play')
spark.sql('select avg(total_plays/total_touchdowns) from nfl_game_play').show()

nfl_team_play = spark.sql('select game_id, posteam_type, count(*) as total_plays, \
                          sum(touchdown) as total_touchdowns from df_nfl group by game_id, posteam_type')

#[Your Code] to take average for total_plays/total_touchdowns by posteam_type
nfl_team_play.createOrReplaceTempView('nfl_team_play')
spark.sql('select posteam_type, avg(total_plays/total_touchdowns) from nfl_team_play group by posteam_type').show()

3. For touchdown, which type happened more likely on average, rush touchdown, pass touchdown or return touchdown? Are the probabilities different by home and away team?

In [ ]:
#DataFrame API Solution
#[Your Code] to calculate total touchdowns and total of each type of touchdowns by game
#then take average for each type of touchdown divided by total touchdowns
nfl_game_play = df_nfl.groupBy('game_id').agg(fn.sum('touchdown').alias('total_touchdowns'),
                                              fn.sum('pass_touchdown').alias('pass_touchdowns'),
                                              fn.sum('rush_touchdown').alias('rush_touchdowns'),
                                              fn.sum('return_touchdown').alias('return_touchdowns'))
nfl_game_play.groupBy().agg(fn.avg(fn.col('pass_touchdowns')/fn.col('total_touchdowns')),
                            fn.avg(fn.col('rush_touchdowns')/fn.col('total_touchdowns')),
                            fn.avg(fn.col('return_touchdowns')/fn.col('total_touchdowns'))).show()

#[Your Code] to calculate total touchdowns and total of each type of touchdowns by game and posteam_type
#then take average for each type of touchdown divided by total touchdowns by posteam_type
nfl_team_play = df_nfl.groupBy('game_id', 'posteam', 'posteam_type').agg(fn.sum('touchdown').alias('total_touchdowns'),
                                                                         fn.sum('pass_touchdown').alias('pass_touchdowns'),
                                                                         fn.sum('rush_touchdown').alias('rush_touchdowns'),
                                                                         fn.sum('return_touchdown').alias('return_touchdowns'))
nfl_team_play.groupBy('posteam_type').agg(fn.avg(fn.col('pass_touchdowns')/fn.col('total_touchdowns')),
                                          fn.avg(fn.col('rush_touchdowns')/fn.col('total_touchdowns')),
                                          fn.avg(fn.col('return_touchdowns')/fn.col('total_touchdowns'))).show()

In [ ]:
#Spark SQL Solution
df_nfl.createOrReplaceTempView('df_nfl')

#[Your Code] to calculate total touchdowns and total of each type of touchdowns by game
#then take average for each type of touchdown divided by total touchdowns
nfl_game_play = spark.sql('select game_id, sum(touchdown) as total_touchdowns, \
                          sum(pass_touchdown) as pass_touchdowns, sum(rush_touchdown) as rush_touchdowns, \
                          sum(return_touchdown) as return_touchdowns from df_nfl group by game_id')
nfl_game_play.createOrReplaceTempView('nfl_game_play')
spark.sql('select avg(pass_touchdowns/total_touchdowns), avg(rush_touchdowns/total_touchdowns), \
          avg(return_touchdowns/total_touchdowns) from nfl_game_play').show()

#[Your Code] to calculate total touchdowns and total of each type of touchdowns by game and posteam_type
#then take average for each type of touchdown divided by total touchdowns by posteam_type
nfl_team_play = spark.sql('select game_id, posteam, posteam_type, sum(touchdown) as total_touchdowns, \
                          sum(pass_touchdown) as pass_touchdowns, sum(rush_touchdown) as rush_touchdowns, \
                          sum(return_touchdown) as return_touchdowns from df_nfl group by game_id, posteam, posteam_type')
nfl_team_play.createOrReplaceTempView('nfl_team_play')
spark.sql('select posteam_type, avg(pass_touchdowns/total_touchdowns), avg(rush_touchdowns/total_touchdowns), \
          avg(return_touchdowns/total_touchdowns) from nfl_team_play group by posteam_type').show()

4. For each calendar year, which team(s) has the highest winning rate?

In [ ]:
#let's look at the available teams
df_nfl.select('home_team').distinct().show(50)

In [ ]:
nfl_game_info = nfl_game_info.withColumn('win_team', fn.when(fn.col('total_home_score') > fn.col('total_away_score'), fn.col('home_team')).otherwise(fn.col('away_team')))
nfl_game_info = nfl_game_info.withColumn('game_year', fn.substring('game_date', 0, 4))
nfl_game_info.show()

In [ ]:
#DataFrame API Solution
#create three sub dataframe, by team-year
win_count = nfl_game_info.groupBy(fn.col('win_team').alias('team'), 'game_year').agg(fn.count('win_team').alias('win_count'))
home_count = nfl_game_info.groupBy(fn.col('home_team').alias('team'), 'game_year').agg(fn.count('home_team').alias('home_count'))
away_count = nfl_game_info.groupBy(fn.col('away_team').alias('team'), 'game_year').agg(fn.count('away_team').alias('away_count'))

#[Your Code] to join the three dataframes for subsequent calculations
team_count = win_count.join(home_count, ['team', 'game_year']).join(away_count, ['team', 'game_year'])
team_count.show()

In [ ]:
#DataFrame API Solution
#generate total game counts and winning rate
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

#[Your Code] to join the three dataframes by 'team' for subsequent calculations by 'team'
window = Window.partitionBy('game_year')
team_count.withColumn("max_win_rate", fn.max("win_rate").over(window)).filter("max_win_rate = win_rate").drop("max_win_rate").show()

In [ ]:
#Spark SQL Solution
nfl_game_info.createOrReplaceTempView('nfl_game_info')

#create three sub dataframe, by team-year
win_count = spark.sql('select win_team as team, game_year, count(*) as win_count from nfl_game_info group by win_team, game_year')
home_count = spark.sql('select home_team as team, game_year, count(*) as home_count from nfl_game_info group by home_team, game_year')
away_count = spark.sql('select away_team as team, game_year, count(*) as away_count from nfl_game_info group by away_team, game_year')

#[Your Code] to join the three dataframes by 'team' for subsequent calculations
win_count.createOrReplaceTempView('win_count')
home_count.createOrReplaceTempView('home_count')
away_count.createOrReplaceTempView('away_count')

spark.sql('select a.*, b.home_count, c.away_count from win_count a, home_count b, away_count c where a.team=b.team and a.team=c.team')
team_count.show()

In [ ]:
#Spark SQL Solution
#generate total game counts and winning rate
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

#[Your Code] to obtain the team(s) with highest winning rate in each calendar year
spark.sql('select a.* from team_count a, (select game_year, max(win_rate) as max_win_rate from team_count group by game_year) b where a.game_year=b.game_year and a.win_rate=b.max_win_rate').show()